## Testing end-to-end

Using this notebook we will run an end-to-end scrape of the USGS, SNOTEL, and Sentinel data for a single river during a single time-span and write the results to GCS.

In [1]:
import json
import sys
import os
sys.path.append('/Users/gisaac/Documents/GitHub/river_flow_notebooks/')
with open("/Users/gisaac/Documents/GitHub/river_flow_notebooks/tests/test_data/test_meta.json") as f:
    data = json.load(f)

In [2]:
from scraping_functions import HydroScraper

In [3]:
from datetime import datetime
h = HydroScraper(datetime(2020, 1, 1), datetime(2020, 1, 1), "/Users/gisaac/Documents/GitHub/river_flow_notebooks/tests/test_data/test_meta.json")

Getting request from USGS
http://waterservices.usgs.gov/nwis/iv/?format=rdb,1.0&sites=01010000&startDT=2020-01-01&endDT=2020-01-01&parameterCd=00060,00065,00045&siteStatus=all
Request finished
Getting request from ASOS
https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=40B&data=tmpf&data=dwpf&data=relh&data=feel&data=sknt&data=sped&data=alti&data=mslp&data=drct&data=ice_accretion_1hr&data=p01m&data=vsby&data=gust&data=skyc1&data=peak_wind_gust&data=snowdepth&year1=2020&month1=1&day1=1&year2=2020&month2=1&day2=3&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=M&trace=T&direct=no&report_type=3&report_type=4


/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["datetime"] = df["datetime"].map(lambda x: old_timezone.localize(datetime.strptime(x, "%Y-%m-%d %H:%M")).astimezone(new_timezone))
/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cfs"] = pd.to_numeric(df['cfs'], errors='coerce')
/Users/gisaac/Documents/GitHub/river_flow_notebooks/scraping_functions.py:66

   station             valid  tmpf  dwpf    relh   feel  sknt  sped   alti  \
0      40B  2020-01-01 00:17  22.5  22.6  100.42      M   0.0  0.00  29.52   
1      40B  2020-01-01 01:16  22.3  23.2       M      M   2.0  2.30  29.48   
2      40B  2020-01-01 02:16  22.1  22.6       M      M   2.0  2.30  29.45   
3      40B  2020-01-01 03:16  22.1  21.9   99.16  22.10   0.0  0.00  29.42   
4      40B  2020-01-01 04:22  21.9  21.7   99.16  21.90   0.0  0.00  29.40   
5      40B  2020-01-01 05:16  21.9  21.4   97.91  21.90   0.0  0.00  29.37   
6      40B  2020-01-01 06:16  21.7  21.4   98.74  21.70   0.0  0.00  29.35   
7      40B  2020-01-01 07:16  21.6  21.6  100.00  21.60   0.0  0.00  29.35   
8      40B  2020-01-01 08:16  21.6  21.7  100.42      M   0.0  0.00  29.34   
9      40B  2020-01-01 09:16  21.6  21.9       M      M   0.0  0.00  29.33   
10     40B  2020-01-01 10:16  21.0  21.0  100.00  21.00   0.0  0.00  29.32   
11     40B  2020-01-01 11:16  18.3  18.1   99.14  18.30   0.0  0

/Users/gisaac/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/gisaac/anaconda3/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [6]:
h.asos_df

,hour_updated,p01m,valid,tmpf,dwpf,ice_accretion_1hr,mslp,drct,sped,alti,relh,sknt,feel,vsby,gust,skyc1,peak_wind_gust,snowdepth,station_id
0,2020-01-01 01:00:00,0.00,2020-01-01 00:17,22.5,22.6,NaN,1002.0,0.0,0.00,29.52,100.42,0.0,NaN,NaN,NaN,M,NaN,NaN,40B
1,2020-01-01 02:00:00,0.25,2020-01-01 01:16,22.3,23.2,NaN,1001.0,NaN,2.30,29.48,NaN,2.0,NaN,NaN,NaN,M,NaN,NaN,40B
2,2020-01-01 03:00:00,0.00,2020-01-01 02:16,22.1,22.6,NaN,999.9,NaN,2.30,29.45,NaN,2.0,NaN,NaN,NaN,M,NaN,NaN,40B
3,2020-01-01 04:00:00,0.25,2020-01-01 03:16,22.1,21.9,NaN,998.9,0.0,0.00,29.42,99.16,0.0,22.10,NaN,NaN,M,NaN,NaN,40B
4,2020-01-01 05:00:00,0.00,2020-01-01 04:22,21.9,21.7,NaN,998.0,0.0,0.00,29.40,99.16,0.0,21.90,NaN,NaN,M,NaN,NaN,40B
5,2020-01-01 06:00:00,0.00,2020-01-01 05:16,21.9,21.4,NaN,997.0,0.0,0.00,29.37,97.91,0.0,21.90,NaN,NaN,M,NaN,NaN,40B
6,2020-01-01 07:00:00,0.00,2020-01-01 06:16,21.7,21.4,NaN,996.6,0.0,0.00,29.35,98.74,0.0,21.70,NaN,NaN,M,NaN,NaN,40B
7,2020-01-01 08:00:00,0.25,2020-01-01 07:16,21.6,21.6,NaN,996.6,0.0,0.00,29.35,100.00,0.0,21.60,NaN,NaN,M,NaN,NaN,40B
8,2020-01-01 09:00:00,0.00,2020-01-01 08:16,21.6,21.7,NaN,996.1,0.0,0.00,29.34,100.42,0.0,NaN,NaN,NaN,M,NaN,NaN,40B
9,2020-01-01 10:00:00,0.76,2020-01-01 09:16,21.6,21.9,NaN,995.7,0.0,0.00,29.33,NaN,0.0,NaN,NaN,NaN,M,NaN,NaN,40B


In [9]:
h.combine_data()

In [12]:
h.joined_df

,hour_updated,p01m,valid,tmpf,dwpf,ice_accretion_1hr,mslp,drct,sped,alti,...,vsby,gust,skyc1,peak_wind_gust,snowdepth,station_id,index,datetime,cfs,height
0,2020-01-01 05:00:00+00:00,0.00,2020-01-01 04:22,21.9,21.7,NaN,998.0,0.0,0.00,29.40,...,NaN,NaN,M,NaN,NaN,40B,1,2020-01-01 05:00:00+00:00,NaN,3.11
1,2020-01-01 06:00:00+00:00,0.00,2020-01-01 05:16,21.9,21.4,NaN,997.0,0.0,0.00,29.37,...,NaN,NaN,M,NaN,NaN,40B,5,2020-01-01 06:00:00+00:00,NaN,3.10
2,2020-01-01 07:00:00+00:00,0.00,2020-01-01 06:16,21.7,21.4,NaN,996.6,0.0,0.00,29.35,...,NaN,NaN,M,NaN,NaN,40B,9,2020-01-01 07:00:00+00:00,NaN,3.10
3,2020-01-01 08:00:00+00:00,0.25,2020-01-01 07:16,21.6,21.6,NaN,996.6,0.0,0.00,29.35,...,NaN,NaN,M,NaN,NaN,40B,13,2020-01-01 08:00:00+00:00,NaN,3.10
4,2020-01-01 09:00:00+00:00,0.00,2020-01-01 08:16,21.6,21.7,NaN,996.1,0.0,0.00,29.34,...,NaN,NaN,M,NaN,NaN,40B,17,2020-01-01 09:00:00+00:00,NaN,3.10
5,2020-01-01 10:00:00+00:00,0.76,2020-01-01 09:16,21.6,21.9,NaN,995.7,0.0,0.00,29.33,...,NaN,NaN,M,NaN,NaN,40B,21,2020-01-01 10:00:00+00:00,NaN,3.09
6,2020-01-01 11:00:00+00:00,0.25,2020-01-01 10:16,21.0,21.0,NaN,995.5,0.0,0.00,29.32,...,NaN,NaN,M,NaN,NaN,40B,25,2020-01-01 11:00:00+00:00,NaN,3.08
7,2020-01-01 12:00:00+00:00,0.25,2020-01-01 11:16,18.3,18.1,NaN,995.7,0.0,0.00,29.33,...,NaN,NaN,M,NaN,NaN,40B,29,2020-01-01 12:00:00+00:00,NaN,3.07
8,2020-01-01 13:00:00+00:00,0.00,2020-01-01 12:16,17.2,18.0,NaN,995.8,0.0,0.00,29.33,...,NaN,NaN,M,NaN,NaN,40B,33,2020-01-01 13:00:00+00:00,942.0,3.06
9,2020-01-01 14:00:00+00:00,0.00,2020-01-01 13:16,19.0,19.8,NaN,996.1,0.0,0.00,29.34,...,NaN,NaN,M,NaN,NaN,40B,37,2020-01-01 14:00:00+00:00,NaN,3.06


In [22]:
if "snotel" in data:
    h.combine_snotel_with_df()
else:
    h.final_df = h.joined_df

## Add Sentinel Image Paths

In [23]:
import pandas as pd

In [24]:
sentinel_df = pd.read_csv("/Users/gisaac/Downloads/sent_data.csv")

In [25]:
mgr = ["13TDE", "11SPB", "50RKQ"]
sentinel_df[sentinel_df["mgrs_tile"].isin(mgr)].to_csv("example1.2.csv")

In [26]:
sentinel_df

,mgrs_tile,base_url,sensing_time,cloud_cover
0,11SPB,gs://gcp-public-data-sentinel-2/tiles/11/S/PB/...,2017-09-01 18:39:59.64 UTC,1.0863
1,50RKQ,gs://gcp-public-data-sentinel-2/tiles/50/R/KQ/...,2017-09-02 03:04:45.62 UTC,84.0477
2,50TQS,gs://gcp-public-data-sentinel-2/tiles/50/T/QS/...,2017-09-02 03:04:45.62 UTC,1.9648
3,12SWF,gs://gcp-public-data-sentinel-2/tiles/12/S/WF/...,2017-09-02 18:03:52.86 UTC,1.4287
4,12RUP,gs://gcp-public-data-sentinel-2/tiles/12/R/UP/...,2017-09-02 18:14:13.4 UTC,92.4166
...,...,...,...,...
3092174,17RLN,gs://gcp-public-data-sentinel-2/tiles/17/R/LN/...,2019-08-17 16:15:21.005 UTC,100.0000
3092175,18FWL,gs://gcp-public-data-sentinel-2/tiles/18/F/WL/...,2019-08-27 14:56:38.222 UTC,100.0000
3092176,49SFR,gs://gcp-public-data-sentinel-2/tiles/49/S/FR/...,2019-08-06 03:19:11.486 UTC,100.0000
3092177,17TQM,gs://gcp-public-data-sentinel-2/tiles/17/T/QM/...,2019-08-17 16:10:31.174 UTC,100.0000


In [33]:
h.combine_sentinel(sentinel_df, "11SMT")
h.final_df

,hour_updated,p01m,valid,tmpf,dwpf,ice_accretion_1hr,mslp,drct,sped,alti,...,index,datetime,cfs,height,sensing_time_x,base_url_x,sensing_time_y,base_url_y,sensing_time,base_url
0,2020-01-01 05:00:00+00:00,0.00,2020-01-01 04:22,21.9,21.7,NaN,998.0,0.0,0.00,29.40,...,1,2020-01-01 05:00:00+00:00,NaN,3.11,NaT,NaN,NaT,NaN,NaT,NaN
1,2020-01-01 06:00:00+00:00,0.00,2020-01-01 05:16,21.9,21.4,NaN,997.0,0.0,0.00,29.37,...,5,2020-01-01 06:00:00+00:00,NaN,3.10,NaT,NaN,NaT,NaN,NaT,NaN
2,2020-01-01 07:00:00+00:00,0.00,2020-01-01 06:16,21.7,21.4,NaN,996.6,0.0,0.00,29.35,...,9,2020-01-01 07:00:00+00:00,NaN,3.10,NaT,NaN,NaT,NaN,NaT,NaN
3,2020-01-01 08:00:00+00:00,0.25,2020-01-01 07:16,21.6,21.6,NaN,996.6,0.0,0.00,29.35,...,13,2020-01-01 08:00:00+00:00,NaN,3.10,NaT,NaN,NaT,NaN,NaT,NaN
4,2020-01-01 09:00:00+00:00,0.00,2020-01-01 08:16,21.6,21.7,NaN,996.1,0.0,0.00,29.34,...,17,2020-01-01 09:00:00+00:00,NaN,3.10,NaT,NaN,NaT,NaN,NaT,NaN
5,2020-01-01 10:00:00+00:00,0.76,2020-01-01 09:16,21.6,21.9,NaN,995.7,0.0,0.00,29.33,...,21,2020-01-01 10:00:00+00:00,NaN,3.09,NaT,NaN,NaT,NaN,NaT,NaN
6,2020-01-01 11:00:00+00:00,0.25,2020-01-01 10:16,21.0,21.0,NaN,995.5,0.0,0.00,29.32,...,25,2020-01-01 11:00:00+00:00,NaN,3.08,NaT,NaN,NaT,NaN,NaT,NaN
7,2020-01-01 12:00:00+00:00,0.25,2020-01-01 11:16,18.3,18.1,NaN,995.7,0.0,0.00,29.33,...,29,2020-01-01 12:00:00+00:00,NaN,3.07,NaT,NaN,NaT,NaN,NaT,NaN
8,2020-01-01 13:00:00+00:00,0.00,2020-01-01 12:16,17.2,18.0,NaN,995.8,0.0,0.00,29.33,...,33,2020-01-01 13:00:00+00:00,942.0,3.06,NaT,NaN,NaT,NaN,NaT,NaN
9,2020-01-01 14:00:00+00:00,0.00,2020-01-01 13:16,19.0,19.8,NaN,996.1,0.0,0.00,29.34,...,37,2020-01-01 14:00:00+00:00,NaN,3.06,NaT,NaN,NaT,NaN,NaT,NaN


In [34]:
h.final_df.dropna(subset=["base_url"])

,hour_updated,p01m,valid,tmpf,dwpf,ice_accretion_1hr,mslp,drct,sped,alti,...,index,datetime,cfs,height,sensing_time_x,base_url_x,sensing_time_y,base_url_y,sensing_time,base_url
14,2020-01-01 19:00:00+00:00,0.0,2020-01-01 18:17,29.1,24.3,NaN,994.8,NaN,4.6,29.3,...,57,2020-01-01 19:00:00+00:00,NaN,3.04,2020-01-01 19:00:00+00:00,gs://gcp-public-data-sentinel-2/tiles/11/S/MT/...,2020-01-01 19:00:00+00:00,gs://gcp-public-data-sentinel-2/tiles/11/S/MT/...,2020-01-01 19:00:00+00:00,gs://gcp-public-data-sentinel-2/tiles/11/S/MT/...


In [35]:
h.bq_connect.upload_file_to_gcs(h.final_df, "023")

In [ ]:
# 